# Using the bottleneck features of a pre-trained network

Use VGG16 as pre-trained model

Only instantiate the convolutional part of the model, everything up to the fully-connected layers

ref: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Load raw data 
X_train = np.load(".\\data\\X_train.npy")
Y_train = np.load(".\\data\\Y_train.npy")
X_test = np.load(".\\data\\X_test.npy")
Y_test = np.load(".\\data\\Y_test.npy")
print (X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)

In [ ]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

In [ ]:
model.summary()

In [ ]:
# X_train after rescale is too big to fit in memory for bottleneck feature training, load 10000 training samples per batch instead.
batch_num = 1
while True:
    start = (batch_num-1) * 10000
    end = start + 10000
    if batch_num < 4:
        X_train_batch = (X_train[start:end, :, :, :] / 255) * 2.0 - 1.0
    else:
        X_train_batch = (X_train[start:, :, :, :] / 255) * 2.0 - 1.0       
    bottleneck_features_train = model.predict(X_train_batch)
    print (bottleneck_features_train.shape)
    np.save('bottleneck_features_train_aug_{}.npy'.format(batch_num), bottleneck_features_train)
    if batch_num == 4:
        break
    batch_num += 1
    del X_train_batch

In [ ]:
X_test = (X_test / 255) * 2.0 - 1.0 
bottleneck_features_validation = model.predict(X_test)
np.save('bottleneck_features_validation_aug.npy', bottleneck_features_validation)

In [ ]:
del X_train, Y_train, X_test, Y_test, X_train_batch, bottleneck_features_train, bottleneck_features_validation

In [ ]:
# Load raw data 
X_train_1 = np.load("bottleneck_features_train_aug_1.npy")
X_train_2 = np.load("bottleneck_features_train_aug_2.npy")
X_train_3 = np.load("bottleneck_features_train_aug_3.npy")
X_train_4 = np.load("bottleneck_features_train_aug_4.npy")
train_data = np.vstack((X_train_1, X_train_2, X_train_3, X_train_4))
train_data.shape

In [ ]:
train_labels = np.load(".\\data\\Y_train.npy")
validation_labels = np.load(".\\data\\Y_test.npy")
print (train_labels.shape, validation_labels.shape)

In [ ]:
validation_data = np.load('bottleneck_features_validation_aug.npy')
print (validation_data.shape)

In [ ]:
top_model_weights_path = 'bottleneck_fc_model_aug.h5'
epochs = 10
batch_size = 64

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))

# add checkpoint to save best model
checkpoint = ModelCheckpoint(top_model_weights_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# check 5 epochs
early_stop = EarlyStopping(monitor='val_acc', patience=5, mode='max')
callbacks_list = [checkpoint, early_stop]

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_labels,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(validation_data, validation_labels), 
                  callbacks=callbacks_list)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.show()
plt.savefig("VGG_bottleneck_acc_history_aug.png")

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.show()
plt.savefig("VGG_bottleneck_loss_history_aug.png")